# Introduction

The purpose of this .ipynb is to compare affinity of the single Tead4 and double Tead4 motif by injecting motif sequences in randomly generated sequences.

# Computational setup

In [1]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

#Packages
import os
import sys
import pandas as pd
import numpy as np
from pybedtools import BedTool
from tqdm import tqdm

# Settings
os.chdir(f'/n/projects/kd2200/publication/rebuttal/analysis/')
pd.set_option('display.max_columns', 100)

# Custom commands
sys.path.insert(0, f'/n/projects/kd2200/publication/bpnet/analysis/scripts/bpnet/scripts/')
from data_format_functions import myround, myfloor, myceiling

# function to return key for any value 
def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
        if val == value: 
            return key 
    return "key doesn't exist"

Using TensorFlow backend.
2024-08-21 09:20:24,561 [INFO] Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-08-21 09:20:24,563 [INFO] Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-08-21 09:20:24,565 [INFO] NumExpr defaulting to 8 threads.


In [2]:
# #Pre-existing variables
fasta_file = f'/n/projects/kd2200/publication/bpnet/fasta/mm10.fa'
model_dir = f'/n/projects/kd2200/publication/bpnet/model/dataspec.yaml_default_fold_5/'
modisco_dir = f'/n/projects/kd2200/publication/bpnet/modisco/'
cwm_scan_all_dir = f'/n/projects/kd2200/publication/bpnet/modisco/profile/dataspec.yaml_default_fold_5/'
pwm_scan_task_dir = f'/n/projects/kd2200/publication/bpnet/analysis/tsv/'
# Determined variables
motifs_of_interest_dict = {#'Tead4': 'Tead4/m0_p0'}
                           'Tead4-Tead4': 'Tead4/m0_p1'}


In [3]:
import pybedtools
import Bio
from Bio.Seq import Seq
from bpnet.extractors import extract_seq

dfi_dict = {}
for t in motifs_of_interest_dict.keys():

    #Import the task
    task, pattern_short = motifs_of_interest_dict[t].split('/')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/cwm_contrib_sel.tsv.gz',sep = '\t')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/pwm_freq_sel.tsv.gz',sep = '\t')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/cwm_freq_sel.tsv.gz',sep = '\t')
    dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/marg_cwm_freq_all.tsv.gz',sep = '\t')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/cwm_contrib_sel_context.tsv.gz',sep = '\t')
    
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/td_cwm_contrib_sel.tsv.gz',sep = '\t')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/td_pwm_freq_sel.tsv.gz',sep = '\t')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/td_cwm_freq_sel.tsv.gz',sep = '\t')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/td_marg_cwm_freq_sel.tsv.gz',sep = '\t')
    #dfi = pd.read_csv(f'/n/projects/kd2200/publication/rebuttal/analysis/tsv/td_cwm_contrib_sel_context.tsv.gz',sep = '\t')

    dfi['row_idx']=range(dfi.shape[0])

    #Get coordinates in Interval format
    coords_interv = [pybedtools.create_interval_from_list([row.seqnames, int(row.start), int(row.end), 
                                                       str(dfi.row_idx), '.', row.strand]) for i,row in dfi.iterrows()] 

    #Extract sequences
    seqs = []
    for i in coords_interv:
        seq = Seq(extract_seq(i, variant = None, fasta_file = fasta_file, one_hot = False).upper())
        if i.strand=='+': #reverse complement based on which strand the motif maps to
            seq = seq.reverse_complement()
        seqs.append(str(seq))

    dfi['sequence'] = seqs
    dfi_dict[t] = dfi    

In [4]:
dfi

,seqnames,start,end,width,strand,motif_id,motif_alt_id,score,p.value,q.value,matched_sequence,peak_region_id,motif,seq,pwm_match_score,pwm_match_score_q,row_idx,sequence
0,chr2,78049760,78049776,17,-,tead4_double,NaN,9.69767,0.000089,NaN,NaN,9568,tead4_double,ACATATTCCAGATACCT,9.529146,0.005714,0,GGTATCTGGAATATGT
1,chr4,46569428,46569444,17,-,tead4_double,NaN,9.51163,0.000097,NaN,NaN,19672,tead4_double,GAATATCTGAGATATCT,9.139072,0.003214,1,GATATCTCAGATATTC
2,chr1,79679687,79679703,17,-,tead4_double,NaN,9.60465,0.000093,NaN,NaN,2377,tead4_double,ACATATTCAGAATATCT,8.784098,0.001786,2,GATATTCTGAATATGT
3,chr11,20456848,20456864,17,+,tead4_double,NaN,10.06980,0.000075,NaN,NaN,54624,tead4_double,AGATACCCCACATATCA,8.653703,0.001429,3,TGATATGTGGGGTATC
4,chr4,18803828,18803844,17,-,tead4_double,NaN,10.09300,0.000074,NaN,NaN,18938,tead4_double,ACATATTTAAAATATCT,8.589107,0.001071,4,GATATTTTAAATATGT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18831,chr19,26709228,26709244,17,-,tead4_double,NaN,13.74420,0.000010,NaN,NaN,86240,tead4_double,GCATTCCAAGCATGACA,0.000000,0.000000,18831,GTCATGCTTGGAATGC
18832,chr19,37341504,37341520,17,+,tead4_double,NaN,11.25580,0.000042,NaN,NaN,86586,tead4_double,GCATGCCTTTCATCCCA,0.000000,0.000000,18832,TGGGATGAAAGGCATG
18833,chrX,38601047,38601063,17,+,tead4_double,NaN,11.72090,0.000033,NaN,NaN,88149,tead4_double,CCATTCCTAGTATTCTA,0.000000,0.000000,18833,TAGAATACTAGGAATG
18834,chrX,42069606,42069622,17,-,tead4_double,NaN,11.06980,0.000047,NaN,NaN,88173,tead4_double,GCATGGCTAGCATGGTA,0.000000,0.000000,18834,ACCATGCTAGCCATGC


In [5]:
# to change the seq column to be considered
dfi.rename(columns = {'sequence':'seqe'}, inplace = True)
dfi.rename(columns = {'seq':'sequence'}, inplace = True)
#dfi.rename(columns = {'sequence':'seqe'}, inplace = True)
#dfi.rename(columns = {'dna2':'sequence'}, inplace = True)
#dfi.rename(columns = {'sequence':'seqe'}, inplace = True)
#dfi.rename(columns = {'dna3':'sequence'}, inplace = True)

In [6]:
# save as csv
#dfi.to_csv(f'new_tead4_double_noerv_withseq.gz', sep = '\t', index = False)
#dfi.to_csv(f'new_tead4_noerv_withseq.gz', sep = '\t', index = False)
dfi

,seqnames,start,end,width,strand,motif_id,motif_alt_id,score,p.value,q.value,matched_sequence,peak_region_id,motif,sequence,pwm_match_score,pwm_match_score_q,row_idx,seqe
0,chr2,78049760,78049776,17,-,tead4_double,NaN,9.69767,0.000089,NaN,NaN,9568,tead4_double,ACATATTCCAGATACCT,9.529146,0.005714,0,GGTATCTGGAATATGT
1,chr4,46569428,46569444,17,-,tead4_double,NaN,9.51163,0.000097,NaN,NaN,19672,tead4_double,GAATATCTGAGATATCT,9.139072,0.003214,1,GATATCTCAGATATTC
2,chr1,79679687,79679703,17,-,tead4_double,NaN,9.60465,0.000093,NaN,NaN,2377,tead4_double,ACATATTCAGAATATCT,8.784098,0.001786,2,GATATTCTGAATATGT
3,chr11,20456848,20456864,17,+,tead4_double,NaN,10.06980,0.000075,NaN,NaN,54624,tead4_double,AGATACCCCACATATCA,8.653703,0.001429,3,TGATATGTGGGGTATC
4,chr4,18803828,18803844,17,-,tead4_double,NaN,10.09300,0.000074,NaN,NaN,18938,tead4_double,ACATATTTAAAATATCT,8.589107,0.001071,4,GATATTTTAAATATGT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18831,chr19,26709228,26709244,17,-,tead4_double,NaN,13.74420,0.000010,NaN,NaN,86240,tead4_double,GCATTCCAAGCATGACA,0.000000,0.000000,18831,GTCATGCTTGGAATGC
18832,chr19,37341504,37341520,17,+,tead4_double,NaN,11.25580,0.000042,NaN,NaN,86586,tead4_double,GCATGCCTTTCATCCCA,0.000000,0.000000,18832,TGGGATGAAAGGCATG
18833,chrX,38601047,38601063,17,+,tead4_double,NaN,11.72090,0.000033,NaN,NaN,88149,tead4_double,CCATTCCTAGTATTCTA,0.000000,0.000000,18833,TAGAATACTAGGAATG
18834,chrX,42069606,42069622,17,-,tead4_double,NaN,11.06980,0.000047,NaN,NaN,88173,tead4_double,GCATGGCTAGCATGGTA,0.000000,0.000000,18834,ACCATGCTAGCCATGC


## Assess the distribution of motif variant frequency.


In [12]:
seq_list = [v.sequence.value_counts() for v in dfi_dict.values()]
seq_df = pd.DataFrame()
for i,l in enumerate(seq_list):
    df = pd.DataFrame(l).reset_index()
    df.columns = ['sequence','frequency']
    df['motif'] = list(motifs_of_interest_dict.keys())[i]
    seq_df = seq_df.append(df)

In [8]:
seq_list = [v.sequence for v in dfi_dict.values()]  # Get unique sequences without counting frequency
seq_df = pd.DataFrame()

for i, seqs in enumerate(seq_list):
    df = pd.DataFrame(seqs, columns=['sequence'])  # Create a DataFrame with sequences
    df['motif'] = list(motifs_of_interest_dict.keys())[i]  # Add the motif name
    seq_df = seq_df.append(df, ignore_index=True)  # Append to the main DataFrame

In [13]:
seq_df

,sequence,frequency,motif
0,GCATGCCTTTAATCCCA,692,Tead4-Tead4
1,ACATGCCTTTAATCCCA,372,Tead4-Tead4
2,GAATTTATGAAATTCCT,65,Tead4-Tead4
3,GCATGCCTTTAATTCCA,37,Tead4-Tead4
4,GCATACCTTTAATCCCA,34,Tead4-Tead4
...,...,...,...
16607,CAATGCTTAGCATTCCT,1,Tead4-Tead4
16608,GCATTCTAGCCATGGTG,1,Tead4-Tead4
16609,ACATTTTATGCATTCAG,1,Tead4-Tead4
16610,GCATACCAAAAATTTTT,1,Tead4-Tead4


In [14]:
seq_summary_df = seq_df.groupby('motif').describe(percentiles = np.arange(0.0, 1.0, 0.1))
seq_summary_df

frequency                                                         \
                count      mean       std  min   0%  10%  20%  30%  40%  50%   
motif                                                                          
Tead4-Tead4   16612.0  1.133879  6.152538  1.0  1.0  1.0  1.0  1.0  1.0  1.0   

                                        
             60%  70%  80%  90%    max  
motif                                   
Tead4-Tead4  1.0  1.0  1.0  1.0  692.0

In [15]:
df1 = seq_df.groupby('motif').quantile(q=0.5)
df1

,frequency
motif,
Tead4-Tead4,1.0


In [16]:
tead_single_df = seq_df[seq_df['motif']=="Tead4-Tead4"]
tead_single_df
tead_single_df_arr = tead_single_df['frequency']
np.percentile(tead_single_df_arr, 50)

1.0

Filter meaningful sequences by requiring that they occur at least 10 times or are in the top 90th percentile of the frequency counts for Tead4. Because Tead4-Tead4 has lot of variability so, we will use 10 times and top 90th percentile together.

In [17]:
seq_filt_df = pd.DataFrame()
for m in seq_df.motif.unique(): 
    df = seq_df[seq_df.motif==m]
    #df = df[(df.frequency>seq_summary_df.loc[m, ('frequency', '90%')]) & (df.frequency >= 5)] #6 for pwms_tdbl 
    df['row'] = range(df.shape[0])
    seq_filt_df = seq_filt_df.append(df)
seq_filt_df.motif.value_counts()

Tead4-Tead4    16612
Name: motif, dtype: int64

In [18]:
seq_filt_df#.frequency.sum()

,sequence,frequency,motif,row
0,GCATGCCTTTAATCCCA,692,Tead4-Tead4,0
1,ACATGCCTTTAATCCCA,372,Tead4-Tead4,1
2,GAATTTATGAAATTCCT,65,Tead4-Tead4,2
3,GCATGCCTTTAATTCCA,37,Tead4-Tead4,3
4,GCATACCTTTAATCCCA,34,Tead4-Tead4,4
...,...,...,...,...
16607,CAATGCTTAGCATTCCT,1,Tead4-Tead4,16607
16608,GCATTCTAGCCATGGTG,1,Tead4-Tead4,16608
16609,ACATTTTATGCATTCAG,1,Tead4-Tead4,16609
16610,GCATACCAAAAATTTTT,1,Tead4-Tead4,16610


# Assess BPNet predictions for variant sequences within each motif

Now, with the variety of sequences that can be selected, assess whether BPNet predicts that some sequences result in higher binding than others. To do this, predict the average signal 256 times. 

## Generate simulated sequences, predict, and summarize for each motif

In [19]:
from bpnet.BPNet import BPNetSeqModel
bpn = BPNetSeqModel.from_mdir(model_dir)

TF-MoDISco is using the TensorFlow backend.


In [20]:
import random
from bpnet.simulate import generate_sim
lower_bounds = 0
upper_bounds = 1000
repeats = 28
random.seed(10)

#For each sequence variant, run BPNet simulate
preds_summary = pd.DataFrame()
for i,row in tqdm(seq_filt_df.iterrows(), total=seq_filt_df.shape[0]):
    motif_variant = row.sequence
    motif = row.motif
    
    pred = bpn.sim_pred(central_motif=motif_variant, side_motif=None, repeat = repeats, side_distances=[])
    pred_summary =  pd.DataFrame.from_dict({k: [np.sum(v[lower_bounds:upper_bounds,:])] for k,v in pred.items()})
    pred_summary.columns = [k.replace('profile/','') for k in pred.keys()]
    pred_summary['sequence'] = motif_variant
    pred_summary['motif'] = motif
    pred_summary['row'] = row.row
    #pred_summary['frequency'] = row.frequency
    preds_summary = preds_summary.append(pred_summary)

100%|██████████| 16612/16612 [10:28<00:00, 26.43it/s]


In [21]:
preds_summary = preds_summary.sort_values(['tead4','yap1'], ascending = False) 


#preds_summary.to_csv(f'new_tead4_double_noervs_preds_summary.csv', index = False)
#preds_summary.to_csv(f'new_tead4_double_noervs_preds_summary_allseq.csv', index = False)
#preds_summary.to_csv(f'new_tead4_double_noervs_preds_summary_allseq_v2.csv', index = False)
#preds_summary.to_csv(f'new_tead4_double_cwms_preds_summary_allseq_genomic_1000_bp_v2.csv', index = False)

#preds_summary.to_csv(f'new_tead4_noervs_preds_summary.csv', index = False)
#preds_summary.to_csv(f'new_tead4_noervs_preds_summary_allseq.csv', index = False)
#preds_summary.to_csv(f'new_tead4_noervs_preds_summary_allseq_v2.csv', index = False)
#preds_summary.to_csv(f'new_tead4_cwms_preds_summary_allseq_genomic_1000bp.csv', index = False)


#preds_summary.to_csv(f'cwm_contrib_sel.csv', index = False)
#preds_summary.to_csv(f'pwm_freq_sel.csv', index = False)
#preds_summary.to_csv(f'cwm_freq_sel.csv', index = False)
#preds_summary.to_csv(f'cwm_contrib_sel_context.csv', index = False)
preds_summary.to_csv(f'marg_cwm_freq_all.csv', index = False)


#preds_summary.to_csv(f'td_cwm_contrib_sel.csv', index = False)
#preds_summary.to_csv(f'td_pwm_freq_sel.csv', index = False)
#preds_summary.to_csv(f'td_cwm_freq_sel.csv', index = False)
#preds_summary.to_csv(f'td_cwm_contrib_sel_context.csv', index = False)
#preds_summary.to_csv(f'td_marg_cwm_freq_sel.csv', index = False)
preds_summary.head()

,cdx2,tfap2c,tead4,yap1,gata3,sequence,motif,row
0,47.674011,15.999769,84.487434,26.559311,15.988951,ACATTCTTTGCATTCCA,Tead4-Tead4,5285
0,52.462959,20.853617,82.714333,27.221485,16.383532,ACATTCTTGGAATTCTT,Tead4-Tead4,7222
0,45.735836,21.094540,76.925552,25.716116,14.091481,AAATTCCTGGAATTCCA,Tead4-Tead4,14218
0,49.480087,21.416353,76.559479,25.545521,15.831800,ACATTCCACGCATTCCA,Tead4-Tead4,12476
0,43.089569,16.997982,76.042038,24.626364,14.666025,ACATTCCTGACATTCCA,Tead4-Tead4,15562
